
# Molecular Visualization Tutorial

## Introduction
Jupyter Notebook can be used for visualization of both molecular and periodic structures.
Multiple viewers have been ported for use in the Notebook - here we will show how to construct a simple molecule and visualize it using several viewers.
This tutorial will also present operations on larger structures, including big PDB files.

To create simple molecular sytems we will use the ASE package.

ASE is an Atomic Simulation Environment written in the Python programming language with the aim of setting up, steering, and analyzing atomistic simulations.

ASE contains objects and structures for atomic structures as well as for calculators.


### This tutorial

In this tutorial we will learn about the basic concepts of visualizing structures using the Atomic Simulation Environment:

1. The Atom and Atoms objects and how to construct atomic structures with and without periodic boundary condistions
2. Visualization of molecules and periodics with x3d and NGLView
2. Visualization of larger structures with NGLView


The tutorial requires that the following python modules are installed:
1. ase
2. matplotlib
3. nglview


These packages should be pre-installed on the virtual machine

### Constructing a water molecule from scratch


For this task we will use ASE. To create molecules we need to first define atoms which are its constituents.

In [ ]:
from ase import Atom
from ase import Atoms

A water molecule is fundametally a terahedral structure with the oxygen atoms in the center and the two hydrogen atoms and two lone pairs at the corners. The angle between two bonds in a fully symmetrical tetrahedron is 109 degrees. The lengths of the O-H bonds can be estimated from the atoms covalent radii. We will place the oxygen in origon and the two hydrogen atoms in the yz plane symmetrically around the z-axis.

In [ ]:
import numpy as np
water = Atoms('OH2') # placing oxygen first

# calculate bond length
from ase.data import covalent_radii
radius_h = covalent_radii[1]  # indexed by atomic number
radius_o = covalent_radii[8]
bondlength = radius_h + radius_o

# calculate x and y projections of unit vector pointing along the o-h bond 
angle = 109.*np.pi/180. # converting to radians
xu = np.cos(angle/2)
yu = np.sin(angle/2)

# set positions

# method 1  (indexig on atoms)
water[1].position = bondlength*np.array([0, xu, yu])
water[2].position = bondlength*np.array([0, xu, -yu])
water.positions

In [ ]:
# A quicker one-liner for numpy users
water.positions[1:, 1:] = bondlength*np.array( [[xu, yu], [xu, -yu]])
water.positions

## Visualization

ASE supports many molecular viewers. For embedded views in Jupyter notebooks it supports two, the x3d  and nglviewer. 

In [ ]:
from ase.visualize import view

The simple ase gui is default, but it pops out as a separate window, if this notebook is run locally.

In [ ]:
view([water, water])

A notebook-embedded representation of a structure can be viewed with the `x3d` viewer, also internal to ASE.

In [ ]:
view(water, viewer='x3d')

The `x3d` viewer does not offer any scripting capability and the only operations you can perform are rotation (left mouse button), translation (Ctrl+left mouse button) and zoom (mouse wheel)

For some more bling directly in the Jupyter notebook we can use the `nglviewer`

In [ ]:
view(water, viewer='ngl')

Alternatively, we may read the ASE structure directly from NGLView.

In [ ]:
import nglview as nv

In [ ]:
v = nv.show_structure_file("dna.pdb")
v

NGLviewer is a powerful utility. We can control many aspects of the display quality.

In [ ]:
# set size of the widget
v._remote_call("setSize", target="Widget", args=["400px", "400px"])

# center the view
v.center()

# change the color of the background
v.background='#ffc'

# modify the z-clipping distance
v.parameters=dict(clipDist=-10)

### Exercise 1: Construct a molecule
H2S has an H-S-H angle of 90 degrees, construct an ASE molecule representing H2S using bond lengths based on covalent radius.

*Hint* See below, in case you don't know the atomic_number of sulfur (but who doesn't?)

In [ ]:
sulfur = Atom('S')
sulfur.number  # atomic number

alternatively

In [ ]:
from ase.data import atomic_numbers
atomic_numbers['S']

In [ ]:
sh2 = 'replace this string with your code'

### Exercise 2 using both x3d and NGLView viewers

Visualize your SH2 molecule:

In [ ]:
'replace with your code'

## Crystals
We can use the viewers to look at not just 0D materials (molecules), but also for periodic systems: 1D (e.g. wires), 2D (e.g. surfaces), and 3D materials (e.g. crystals)

Here we will focus on crystals

Let's read in the NaH structure from a file, already present in the right location.

In [ ]:
from ase import io
nah = io.read('NaH.cif')
view(nah, viewer='ngl')

Repeating the cell three times is as easy as using a simple method on the loaded structure.

In [ ]:
view(nah.repeat(3), viewer='ngl')

### Building
Like for the molecule a crystal can be generated by building from scratch, or reading it from a file as above, or by using predefined structures.
Let's build a crystal for silver using the ASE `bulk` module

In [ ]:
from ase.build import bulk
ag = bulk('Ag')

Note, that ASE automatically assigned crystal symmetry (fcc) and lattice constant. 

This structure can now be nicely visualized.

In [ ]:
view(ag, viewer='ngl')

### Databases

Multiple databases can be queried for systems so we don't need to manually create them!

One example is ASE's own `builder` database

In [ ]:
from ase import build
ch3nh2 = build.molecule('CH3NO2')
view(ch3nh2, viewer='ngl')

Jupyter Notebook can also be used to visualize entries from external databases, like the PDB database. We can query them with the name.
Let's have a look at the main proteaze of the 2019-nCoV coronavirus.

In [ ]:
import nglview
import os
os.environ['HTTPS_PROXY']='http://172.18.12.30:8123'
os.environ['HTTP_PROXY']='http://172.18.12.30:8123'

In [ ]:
# This command will query the online database for the given PDB ID
view = nglview.show_pdbid("6lu7")

In [ ]:
view.render_image()
view

Notice how clicking on the protein shows the clicked atom exact location (residue).

NGLView offers a large number of options to allow for customized view.

In [ ]:
view.add_cartoon(selection="protein")
view.add_surface(selection="protein", opacity=0.3)

# specify color
view.add_cartoon(selection="protein", color='blue')
view.camera = 'orthographic'
view.background = 'yellow'

We can of course load local files for viewing as well.

In [ ]:
view = nglview.show_structure_file("dna.pdb")

In [ ]:
view.add_cartoon()

In [ ]:
view